# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [1]:
!wget "http://www.gutenberg.org/files/2600/2600-0.txt"

--2019-10-06 16:50:51--  http://www.gutenberg.org/files/2600/2600-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3359549 (3.2M) [text/plain]
Saving to: ‘2600-0.txt.1’

2600-0.txt.1        100%[===================>]   3.20M  4.58MB/s    in 0.7s    

2019-10-06 16:50:55 (4.58 MB/s) - ‘2600-0.txt.1’ saved [3359549/3359549]



In [2]:
text = open('2600-0.txt', 'r', encoding='utf-8').read()[2:]
len(text)

3227579

In [0]:
import re

def preprocess_text(text):
    punct = '!"#$%&\'()*+,-—/:;<=>?@[\\]^_`{|}~«»”’“‘'
    regex = re.compile('[%s]' % re.escape(punct))
    t = text.lower()    
    t = regex.sub(' ', t)
    t = re.sub(r'\s+', r' ', t)
    return t

text = preprocess_text(text)
assert len(text) == 3141169

In [0]:
text = text.split('.')
text = [x.strip() for x in text]

In [5]:
print(len(set(text)))

26602


In [0]:
def code_text(text, stoi):
    """
    Encodes symbols with ids
    :param text: list if lists with strings
    :param stoi: dict, character:character_id
    :return: encoded text, list of lists of integers
    """
    
    coded_text = []
    for sent in text:
        coded_sent = []
        for char in sent:
            try:
                coded_sent.append(stoi[char])
            except KeyError:
                print(char)
        coded_text.append(coded_sent)
    return coded_text

In [0]:
def replace_token(token, _id, text):
    """
    Replace token with new id in text
    :param token: tuple
    :param _id: int, new id
    :param text: list of list of ids
    :return: list of list of ids, modified text
    """
    
    text_new = [sent[:] for sent in text]
    for i, sent in enumerate(text):
        modified = 0
        for j, (v, w) in enumerate(zip(sent[:-1], sent[1:])):
            if (v, w) == token:
                text_new[i][j - modified] = _id
                del text_new[i][j - modified + 1]
                modified += 1
    return text_new

In [0]:
from collections import Counter

def count_ngrams(text, n):
    """
    Counts frequency of ngrams

    :param text: list of lists

    :return: Counter of ngrams in text
    """
    bigrams = []
    for sent in text:
        for i in range(len(sent) - (n - 1)):
            bigrams.append(tuple((sent[i: i + n])))

    counted_bigr = Counter(bigrams)

    return counted_bigr

In [0]:
from sklearn.base import TransformerMixin
import nltk
from collections import Counter


class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}

        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """    

        # tokenize text by symbols and fill in self.itos and self.stoi
        self.itos = list(set(''.join(text)))
        
        self.stoi = {}
        for i in range(len(self.itos)):
            self.stoi[self.itos[i]] = i

        bigr = count_ngrams(text, 2)
        text = code_text(text, self.stoi)

        while len(self.itos) < self.vocab_size:
            bigrams = count_ngrams(text, 2)
            most_comm = bigrams.most_common(1)
            if most_comm != []:
                new_token = bigrams.most_common(1)[0][0] # most common bigram
                new_id = len(self.itos)
                self.itos.append(new_token)
                self.stoi[new_token] = new_id
                text = replace_token(new_token, new_id, text)
            else:
                break

        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        # tokenize text by symbols using self.stoi
        text = code_text(text, self.stoi)
 

        for _id, token in enumerate(self.itos):
            text = replace_token(token, _id, text)

        return text
    
    def decode_token(self, _id):
        """
        decodes token
        :param _id: int or tuple
        """

        def lookup(_id):
            if type(_id) is int:
                token = self.itos[_id]
                if type(token) is str:
                    return token
                else:
                    return lookup(token)
            else: # _id is tuple
                return lookup(_id[0]) + lookup(_id[1])

        return lookup(_id)
              
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
                


In [116]:
from time import time
s = time()
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)
print(time() - s)

109.44263529777527


In [117]:
assert bpe.decode(tokenized_text[0]) == text[0]
print('OK')

OK


In [0]:
def laplace_smoothing(n, d=1, tau=1):
  """
  Counts Laplace smoothing

  :param n: int
  :param d: int or float, delta
  tau: int or float
  """
  return (n + d) ** (1 / tau)

In [0]:
import numpy as np
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = None
        self.start_token = vocab_size
        self.end_token = vocab_size + 1
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = [self.get_proba(a, b, i, tau) for i in range(self.vocab_size)]
        return result
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        probs = []

        for i in range(self.vocab_size):
            p = self.proba[(a, b, i)]
            probs.append(laplace_smoothing(p))
    
        p_trigram = self.proba[(a, b, c)]
        result = laplace_smoothing(p_trigram) / sum(probs)
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        text = [[self.start_token] + sent + [self.end_token] for sent in text]
        self.proba = count_ngrams(text, 3)
        
        return self
    


In [0]:
lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
def reform_results(l, k):
    return sorted(enumerate(l), key=lambda x: x[1], reverse=True)[:k]

In [0]:
from math import log

def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    if len(input_seq) < 2:
        input_seq = [lm.start_token] + input_seq

    a, b = input_seq[-2:]
    most_probable = lm.infer(a, b, tau) 
    
    beam = []
    for token, proba in reform_results(most_probable, k):
        beam.append(([b, token], log(proba))) # store in beam tuples of current sequences and their log probabilities
    
    for i in range(max_len):
        candidates = []
        candidates_proba = []
    
        for sent, sent_proba in beam:
            if sent[-1] == lm.end_token:# TODO process terminal token 
                continue
            else:
                c, d = sent[-2:]
                proba = lm.infer(c, d, tau)
                best_k = reform_results(proba, k)
                candidates += [sent + [token] for token, proba in best_k]
                candidates_proba += [sent_proba + log(proba) for token, proba 
                                     in best_k]  
      
        beam = [(candidates[index], log_proba) for index, log_proba in reform_results(candidates_proba, k)]
    return [(sent[1:], exp(sent_proba)) for sent, sent_proba in beam]

In [0]:
def print_decoded(result, bpe, end):
    for snt, snt_proba in result:
        if snt[-1] == end:
            snt = snt[:-1]
        print(f'Сгенерировалось "{bpe.decode(snt)}"')
        print(f'Вероятность: {snt_proba}')

In [250]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
print_decoded(result, bpe, lm.end_token)

Сгенерировалось "with himself s"
Вероятность: 1.8281613740824351e-06
Сгенерировалось "with himself and "
Вероятность: 1.6490761782539504e-06
Сгенерировалось "with himself i"
Вероятность: 1.6117667624563522e-06
Сгенерировалось "with himself he "
Вероятность: 1.4625290992659495e-06
Сгенерировалось "with himself y"
Вероятность: 1.3506008518731458e-06
Сгенерировалось "with himself in"
Вероятность: 1.3506008518731458e-06
Сгенерировалось "with himself b"
Вероятность: 1.149130006566103e-06
Сгенерировалось "with himself a"
Вероятность: 1.0446636423328202e-06
Сгенерировалось "with himself c"
Вероятность: 9.84968577056659e-07
Сгенерировалось "with himself f"
Вероятность: 9.625829275780997e-07


In [251]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
print_decoded(result, bpe, lm.end_token)

Сгенерировалось "e with himself "
Вероятность: 1.3426008715224258e-05
Сгенерировалось "e with himself"
Вероятность: 2.60973939897802e-06
Сгенерировалось "e with himself w"
Вероятность: 1.6769409792629846e-06
Сгенерировалось "e with himselve"
Вероятность: 1.5403514317585013e-06
Сгенерировалось "e with his fack"
Вероятность: 1.2157350103734083e-06
Сгенерировалось "e with his face "
Вероятность: 1.176603817647416e-06
Сгенерировалось "e with himself to "
Вероятность: 1.037607230918971e-06
Сгенерировалось "e with his heas"
Вероятность: 9.673468884321173e-07
Сгенерировалось "e with his fact"
Вероятность: 9.667183293898269e-07
Сгенерировалось "e with his hear "
Вероятность: 8.910232584258684e-07


In [252]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
print_decoded(result, bpe, lm.end_token)

Сгенерировалось "ed to himself you"
Вероятность: 3.9972382872566104e-06
Сгенерировалось "ed to himself in "
Вероятность: 1.9750752458246372e-06
Сгенерировалось "ed to himself it "
Вероятность: 1.7021541990555776e-06
Сгенерировалось "ed to himself i "
Вероятность: 1.4922512460114162e-06
Сгенерировалось "ed to himself be"
Вероятность: 9.330541012668513e-07
Сгенерировалось "ed to himself sa"
Вероятность: 9.303074810315902e-07
Сгенерировалось "ed to himself bu"
Вероятность: 8.981497051428168e-07
Сгенерировалось "ed to himself he ha"
Вероятность: 8.943145534674896e-07
Сгенерировалось "ed to himself he w"
Вероятность: 8.367494787615365e-07
Сгенерировалось "ed to himself is "
Вероятность: 8.060978953040398e-07


In [253]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
print_decoded(result, bpe, lm.end_token)

Сгенерировалось "prince and their"
Вероятность: 1.6566138988035323e-06
Сгенерировалось "prince andressi"
Вероятность: 1.5657239895586834e-06
Сгенерировалось "prince with his "
Вероятность: 1.2917322251807314e-06
Сгенерировалось "prince andrew s"
Вероятность: 1.0067437466727315e-06
Сгенерировалось "prince andrew b"
Вероятность: 8.106248349832388e-07
Сгенерировалось "prince andrew and "
Вероятность: 7.714010526453404e-07
Сгенерировалось "prince and they w"
Вероятность: 7.202054718631012e-07
Сгенерировалось "prince andrew c"
Вероятность: 7.125653791384926e-07
Сгенерировалось "prince andrew h"
Вероятность: 7.060280820821751e-07
Сгенерировалось "prince andrew s "
Вероятность: 7.060280820821751e-07


In [254]:
from math import exp, log

def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """

    n = len(snt)
    p = 0

    for trigram in count_ngrams([snt], 3):
      p += log(lm.get_proba(*trigram))
    result = exp(p) ** (-1 / n) 

    return result

perplexity(tokenized_text[0], lm)

11.989877698946975